In [12]:
# pip install html5lib

In [13]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import pickle

In [14]:
# URL of the QB earnings data:
url = "https://overthecap.com/career-earnings/quarterback"

# Send a GET request to the URL
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    # Parse the HTML content
    soup = BeautifulSoup(response.content, 'html5lib')
    # Find the table
    table = soup.find('table')
    # Read table into a pandas DataFrame
    df = pd.read_html(str(table))[0]

else:
    print("Failed to retrieve the page")

# Display the DataFrame
display(df.head())
display(df.info())

,Player,Years Active,Career Earnings,Cap-Inflated Earnings,Earnings By Team
0,Aaron Rodgers,2005‑,"$343,531,094","$546,133,077","Packers: $306,642,206 Jets: $36,888,888"
1,Matt Stafford,2009‑,"$328,000,000","$507,909,318","Lions: $219,000,000 Rams: $109,000,000"
2,Tom Brady,2000‑2022,"$317,619,794","$600,482,947","Patriots: $230,324,206 Buccaneers: $87,295,588"
3,Matt Ryan,2008‑2022,"$306,205,882","$501,677,687","Falcons: $269,500,000 Colts: $36,705,882"
4,Russell Wilson,2012‑,"$305,340,123","$408,432,196","Seahawks: $181,340,123 Broncos: $124,000,000"


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 391 entries, 0 to 390
Data columns (total 5 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   Player                 391 non-null    object
 1   Years Active           391 non-null    object
 2   Career Earnings        391 non-null    object
 3   Cap-Inflated Earnings  391 non-null    object
 4   Earnings By Team       391 non-null    object
dtypes: object(5)
memory usage: 15.4+ KB


None

In [15]:
df_clean = df.copy()

df_clean['Player'] = df_clean['Player'].astype(str)
df_clean['Years Active'] = df_clean['Years Active'].astype(str)

# this is a weird 'short' dash separating the years...had to cut and paste from the output
# Just split once (n=1)
df_clean[['Year_Drafted', 'Year_Final']] = df_clean['Years Active'].str.split('‑', n=1, expand=True)

display(df_clean['Year_Drafted'].value_counts())
display(df_clean['Year_Final'].value_counts())

Year_Drafted
9999    86
2017    23
2019    22
2023    20
2024    20
2013    20
2020    18
2018    18
2016    18
2015    16
2022    15
2014    15
2021    15
2011    14
2012    13
2005     7
2008     5
2009     5
2004     5
2007     4
2006     4
2010     4
2002     3
1995     2
2000     2
1997     2
1993     2
1983     2
2003     2
1998     2
1999     2
2001     2
1991     1
1985     1
1989     1
Name: count, dtype: int64

Year_Final
        106
0        86
2023     24
2022     24
2020     21
2017     20
2015     18
2018     15
2021     14
2016     13
2019     12
2013     11
2012      5
2011      4
2014      4
2010      3
2003      2
2006      2
1999      2
2009      2
1998      1
2000      1
2007      1
Name: count, dtype: int64

In [16]:
# Convert to numeric, setting errors='coerce' to handle non-numeric values
df_clean['Year_Drafted'] = pd.to_numeric(df_clean['Year_Drafted'], errors='coerce')
df_clean['Year_Final'] = pd.to_numeric(df_clean['Year_Final'], errors='coerce')

# Count NaN values
nan_drafted = df_clean['Year_Drafted'].isna().sum()
nan_final = df_clean['Year_Final'].isna().sum()

# Count 0 values
zero_drafted = (df_clean['Year_Drafted'] == 0).sum()
zero_final = (df_clean['Year_Final'] == 0).sum()

# Count non-numerical values (already handled by pd.to_numeric with errors='coerce')
non_numerical_drafted = df_clean['Year_Drafted'].isna().sum()
non_numerical_final = df_clean['Year_Final'].isna().sum()

print(f"Year_Drafted: \t({nan_drafted}) NaN-values;\
\t\t({zero_drafted}) 0-values; \t({non_numerical_drafted}): Non-numerical-values.")
print(f"Year_Final: \t({nan_final}) NaN-values;\
\t({zero_final}) 0-values; \t({non_numerical_final}): Non-numerical-values.")

Year_Drafted: 	(0) NaN-values;		(0) 0-values; 	(0): Non-numerical-values.
Year_Final: 	(106) NaN-values;	(86) 0-values; 	(106): Non-numerical-values.


In [17]:
# Need to cleanup the Year_Final column:
# Replace NaNs and 0s with 9999
df_clean['Year_Final'].fillna(9999, inplace=True)
df_clean['Year_Final'].replace(0, 9999, inplace=True)
# Convert the column to integers
df_clean['Year_Final'] = df_clean['Year_Final'].astype(int)

# Count NaN values
nan_drafted = df_clean['Year_Drafted'].isna().sum()
nan_final = df_clean['Year_Final'].isna().sum()

# Count 0 values
zero_drafted = (df_clean['Year_Drafted'] == 0).sum()
zero_final = (df_clean['Year_Final'] == 0).sum()

# Count non-numerical values (already handled by pd.to_numeric with errors='coerce')
non_numerical_drafted = df_clean['Year_Drafted'].isna().sum()
non_numerical_final = df_clean['Year_Final'].isna().sum()

print(f"Year_Drafted: \t({nan_drafted}) NaN-values;\
\t({zero_drafted}) 0-values; \t({non_numerical_drafted}): Non-numerical-values.")
print(f"Year_Final: \t({nan_final}) NaN-values;\
\t({zero_final}) 0-values; \t({non_numerical_final}): Non-numerical-values.")

Year_Drafted: 	(0) NaN-values;	(0) 0-values; 	(0): Non-numerical-values.
Year_Final: 	(0) NaN-values;	(0) 0-values; 	(0): Non-numerical-values.


In [18]:
display(df_clean.head())
display(df_clean.info())

,Player,Years Active,Career Earnings,Cap-Inflated Earnings,Earnings By Team,Year_Drafted,Year_Final
0,Aaron Rodgers,2005‑,"$343,531,094","$546,133,077","Packers: $306,642,206 Jets: $36,888,888",2005,9999
1,Matt Stafford,2009‑,"$328,000,000","$507,909,318","Lions: $219,000,000 Rams: $109,000,000",2009,9999
2,Tom Brady,2000‑2022,"$317,619,794","$600,482,947","Patriots: $230,324,206 Buccaneers: $87,295,588",2000,2022
3,Matt Ryan,2008‑2022,"$306,205,882","$501,677,687","Falcons: $269,500,000 Colts: $36,705,882",2008,2022
4,Russell Wilson,2012‑,"$305,340,123","$408,432,196","Seahawks: $181,340,123 Broncos: $124,000,000",2012,9999


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 391 entries, 0 to 390
Data columns (total 7 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   Player                 391 non-null    object
 1   Years Active           391 non-null    object
 2   Career Earnings        391 non-null    object
 3   Cap-Inflated Earnings  391 non-null    object
 4   Earnings By Team       391 non-null    object
 5   Year_Drafted           391 non-null    int64 
 6   Year_Final             391 non-null    int32 
dtypes: int32(1), int64(1), object(5)
memory usage: 20.0+ KB


None

In [19]:
df_clean_2 = df_clean.copy()
df_clean_2["Career Earnings (millions)"] = round(df_clean_2["Career Earnings"].     \
                                                str.replace('$', '').str.replace(',','').astype(int)/1_000_000, 3)
df_clean_2["Cap-Inflated Earnings (millions)"] = round(df_clean_2["Cap-Inflated Earnings"]. \
                                                str.replace('$', '').str.replace(',','').astype(int)/1_000_000, 3)

display(df_clean_2["Career Earnings (millions)"].nsmallest(10))
display(df_clean_2["Cap-Inflated Earnings (millions)"].nsmallest(10))
display(df_clean_2.head())
display(df_clean_2.info())

389    0.000
390    0.000
385    0.001
386    0.001
387    0.001
388    0.001
379    0.002
380    0.002
381    0.002
382    0.002
Name: Career Earnings (millions), dtype: float64

390    0.000
386    0.001
387    0.001
389    0.001
384    0.002
385    0.002
388    0.002
376    0.003
380    0.003
381    0.003
Name: Cap-Inflated Earnings (millions), dtype: float64

,Player,Years Active,Career Earnings,Cap-Inflated Earnings,Earnings By Team,Year_Drafted,Year_Final,Career Earnings (millions),Cap-Inflated Earnings (millions)
0,Aaron Rodgers,2005‑,"$343,531,094","$546,133,077","Packers: $306,642,206 Jets: $36,888,888",2005,9999,343.531,546.133
1,Matt Stafford,2009‑,"$328,000,000","$507,909,318","Lions: $219,000,000 Rams: $109,000,000",2009,9999,328.000,507.909
2,Tom Brady,2000‑2022,"$317,619,794","$600,482,947","Patriots: $230,324,206 Buccaneers: $87,295,588",2000,2022,317.620,600.483
3,Matt Ryan,2008‑2022,"$306,205,882","$501,677,687","Falcons: $269,500,000 Colts: $36,705,882",2008,2022,306.206,501.678
4,Russell Wilson,2012‑,"$305,340,123","$408,432,196","Seahawks: $181,340,123 Broncos: $124,000,000",2012,9999,305.340,408.432


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 391 entries, 0 to 390
Data columns (total 9 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   Player                            391 non-null    object 
 1   Years Active                      391 non-null    object 
 2   Career Earnings                   391 non-null    object 
 3   Cap-Inflated Earnings             391 non-null    object 
 4   Earnings By Team                  391 non-null    object 
 5   Year_Drafted                      391 non-null    int64  
 6   Year_Final                        391 non-null    int32  
 7   Career Earnings (millions)        391 non-null    float64
 8   Cap-Inflated Earnings (millions)  391 non-null    float64
dtypes: float64(2), int32(1), int64(1), object(5)
memory usage: 26.1+ KB


None

In [20]:
# Extract team names and earnings using extactall to extract all matches of the pattern

# Extract team names and earnings
team_earnings = df_clean_2['Earnings By Team'].str.extractall(r'(?P<Team>\w+): \$(?P<Earnings>[\d,]+)')
team_earnings['Earnings'] = round(team_earnings['Earnings'].str.replace(',', '').astype(float)/1_000_000, 3)

# display(team_earnings['Team'])
# display(team_earnings['Earnings'])

# Group by the original index and convert to dictionaries
earnings_dict = team_earnings.groupby(level=0).apply(lambda x: dict(zip(x['Team'], x['Earnings'])))

display(earnings_dict)
# display(team_earnings.info())
# Pivot the extracted data
team_earnings = team_earnings.reset_index().pivot(index='level_0', columns='Team', values='Earnings')

# Add the dictionary column to the original DataFrame
df_clean_2['Earnings By Team Dict'] = earnings_dict
display(df_clean_2.head())
display(df_clean_2.info())
display(df_clean_2.columns)

0             {'Packers': 306.642, 'Jets': 36.889}
1                  {'Lions': 219.0, 'Rams': 109.0}
2      {'Patriots': 230.324, 'Buccaneers': 87.296}
3              {'Falcons': 269.5, 'Colts': 36.706}
4           {'Seahawks': 181.34, 'Broncos': 124.0}
                          ...                     
386                             {'Cowboys': 0.001}
387                              {'Browns': 0.001}
388                          {'Commanders': 0.001}
389                               {'Broncos': 0.0}
390                                 {'49ers': 0.0}
Length: 391, dtype: object

,Player,Years Active,Career Earnings,Cap-Inflated Earnings,Earnings By Team,Year_Drafted,Year_Final,Career Earnings (millions),Cap-Inflated Earnings (millions),Earnings By Team Dict
0,Aaron Rodgers,2005‑,"$343,531,094","$546,133,077","Packers: $306,642,206 Jets: $36,888,888",2005,9999,343.531,546.133,"{'Packers': 306.642, 'Jets': 36.889}"
1,Matt Stafford,2009‑,"$328,000,000","$507,909,318","Lions: $219,000,000 Rams: $109,000,000",2009,9999,328.000,507.909,"{'Lions': 219.0, 'Rams': 109.0}"
2,Tom Brady,2000‑2022,"$317,619,794","$600,482,947","Patriots: $230,324,206 Buccaneers: $87,295,588",2000,2022,317.620,600.483,"{'Patriots': 230.324, 'Buccaneers': 87.296}"
3,Matt Ryan,2008‑2022,"$306,205,882","$501,677,687","Falcons: $269,500,000 Colts: $36,705,882",2008,2022,306.206,501.678,"{'Falcons': 269.5, 'Colts': 36.706}"
4,Russell Wilson,2012‑,"$305,340,123","$408,432,196","Seahawks: $181,340,123 Broncos: $124,000,000",2012,9999,305.340,408.432,"{'Seahawks': 181.34, 'Broncos': 124.0}"


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 391 entries, 0 to 390
Data columns (total 10 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   Player                            391 non-null    object 
 1   Years Active                      391 non-null    object 
 2   Career Earnings                   391 non-null    object 
 3   Cap-Inflated Earnings             391 non-null    object 
 4   Earnings By Team                  391 non-null    object 
 5   Year_Drafted                      391 non-null    int64  
 6   Year_Final                        391 non-null    int32  
 7   Career Earnings (millions)        391 non-null    float64
 8   Cap-Inflated Earnings (millions)  391 non-null    float64
 9   Earnings By Team Dict             391 non-null    object 
dtypes: float64(2), int32(1), int64(1), object(6)
memory usage: 29.1+ KB


None

Index(['Player', 'Years Active', 'Career Earnings', 'Cap-Inflated Earnings',
       'Earnings By Team', 'Year_Drafted', 'Year_Final',
       'Career Earnings (millions)', 'Cap-Inflated Earnings (millions)',
       'Earnings By Team Dict'],
      dtype='object')

In [21]:
df_final = df_clean_2[['Player', 'Year_Drafted', 'Year_Final',
       'Career Earnings (millions)', 'Cap-Inflated Earnings (millions)',
       'Earnings By Team Dict']]

df_final.head()

,Player,Year_Drafted,Year_Final,Career Earnings (millions),Cap-Inflated Earnings (millions),Earnings By Team Dict
0,Aaron Rodgers,2005,9999,343.531,546.133,"{'Packers': 306.642, 'Jets': 36.889}"
1,Matt Stafford,2009,9999,328.000,507.909,"{'Lions': 219.0, 'Rams': 109.0}"
2,Tom Brady,2000,2022,317.620,600.483,"{'Patriots': 230.324, 'Buccaneers': 87.296}"
3,Matt Ryan,2008,2022,306.206,501.678,"{'Falcons': 269.5, 'Colts': 36.706}"
4,Russell Wilson,2012,9999,305.340,408.432,"{'Seahawks': 181.34, 'Broncos': 124.0}"


In [22]:
#Export csv
csv_path = "../Data_Artifacts/otcap_career_earnings.csv"
df_final.to_csv(csv_path, index=False)

###  ** ALL REMAINING CELLS: NOT USED **

In [ ]:
# Import CSV
# df = pd.read_csv(csv_path)

In [139]:
# Save as Pickle; pickle.dump to store data
pickle_path = 'Resources/qb_earnings_data_clean.pkl'
with open(pickle_path, 'wb') as file:
    pickle.dump(df, file)

In [ ]:
# Reload Dataframe from pkl file, with rb 'read binary'
# with open(pickle_path', 'rb') as file:
#     df = pickle.load(file)